In [2]:
import numpy as np
import scipy
import pandas as pd

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.exceptions import FitFailedWarning

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import ensemble

import joblib
import warnings

In [ ]:
# full code on GitHub: https://github.com/Sepuliini/DDO-course

In [4]:
# We will start by loading the dataset and exploring its properties. 

In [5]:
# Load dataset
data_full = '/Users/sepuliini/Desktop/DDO_kurssi/Metall. Application.xls'

# Load datasets into pandas DataFrame
data = pd.read_excel(data_full, header = 1)
data.head()

,Nb,Ti,V,DATA NO,Paper no.,C,Si,Mn,P,S,...,charpy energy J (-80 C),energy at 19'C (J),50%FATT 'C,Fracture toughness (J(ic)),Pearlite %,type,hardness(BHN),Room temp impact energy J,USE(J),shelf energy J
0,0,0,0,5,E19,0.14,0.35,1.56,0.01,0.015,...,NaN,NaN,NaN,NaN,26+-1,F/P,142+-2,79+-6,NaN,NaN
1,0,0,0.096,6,E19,0.15,0.31,1.41,0.01,0.015,...,NaN,NaN,NaN,NaN,26+-1,F/P,181+-3,20+-5,NaN,NaN
2,0,0.075,0,9,h 1,0.054,0.03,0.54,0,0,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
3,0,0.092,0,10,h 1,0.061,0.03,0.56,0,0,...,NaN,NaN,NaN,NaN,<0.5,NaN,NaN,NaN,NaN,NaN
4,0,0.092,0,11,h 1,0.055,0.03,0.55,0,0,...,NaN,NaN,NaN,NaN,<0.5,NaN,NaN,NaN,NaN,NaN


In [6]:
# All of the rows were not relevant for this assignment so I removed those columns. 

In [7]:
columns_of_interest = list(range(3)) + list(range(5, 25)) + [29, 30, 31]
# Select only the columns of interest
filtered_data = data.iloc[:-4, columns_of_interest]
filtered_data.head()

,Nb,Ti,V,C,Si,Mn,P,S,Mo,Ni,...,Cr,Ce,Cu,Ca,Zr,O,Cb,YS(Mpa),UTS(Mpa),%EL
0,0,0,0,0.14,0.35,1.56,0.01,0.015,0,0,...,0,0,0,0,0,0,0,272,470,30
1,0,0,0.096,0.15,0.31,1.41,0.01,0.015,0,0,...,0,0,0,0,0,0,0,378,579,23
2,0,0.075,0,0.054,0.03,0.54,0,0,0,0,...,0,0,0,0,0,0,0,412,NaN,NaN
3,0,0.092,0,0.061,0.03,0.56,0,0,0,0,...,0,0,0,0,0,0,0,410,NaN,NaN
4,0,0.092,0,0.055,0.03,0.55,0,0,0,0,...,0,0,0,0,0,0,0,395,NaN,NaN


In [8]:
# After this I went trough the dataset and checked that it had all numeric values

In [9]:
# Check for non-numeric entries in each column
for column in filtered_data.columns:
    # Convert to numeric, forcing non-convertible values to NaN
    numeric_data = pd.to_numeric(filtered_data[column], errors='coerce')
    
    # Find non-numeric entries
    non_numeric_entries = filtered_data[column][numeric_data.isna() & filtered_data[column].notna()]

    # Display unique non-numeric entries from this column
    if not non_numeric_entries.empty:
        print(f"Non-numeric entries in column '{column}':")
        print(non_numeric_entries.unique())

Non-numeric entries in column 'C':
['.030-.1']
Non-numeric entries in column 'Mn':
['.5-1.5']
Non-numeric entries in column 'S':
['<0.003']
Non-numeric entries in column 'Al':
['.02-.05']
Non-numeric entries in column 'V.1':
['.015-.03' '.025-.035' '.045-.055' '.055-.065' '.075-.085']
Non-numeric entries in column 'UTS(Mpa)':
['460-610' '720-870' '~325']
Non-numeric entries in column '%EL':
['~5']


In [10]:
# Some of the values were not numeric so I made a simple script to convert these

In [11]:
def convert_value(val):
    try:
        # First, ensure that we're working with a string. If it's not a string, return as is.
        if pd.isna(val):
            return np.nan  # Handle NaN values immediately
        if isinstance(val, str):  # Ensure the value is a string before checking with string methods
            if '-' in val:
                low, high = map(float, val.split('-'))
                return (low + high) / 2  # Return the midpoint of the range
            elif val.startswith('<') or val.startswith('~'):
                return float(val[1:])  # Remove the prefix and convert to float
        return float(val)  # Convert numeric strings to floats
    except ValueError:
        return np.nan  # Return NaN for values that cannot be converted to float
    except Exception as e:
        print(f"Error converting {val}: {e}")
        return np.nan


In [12]:
# List of columns to apply conversion
columns_to_convert = ['C', 'Mn', 'S', 'Al', 'V.1', 'UTS(Mpa)', '%EL']
# Apply conversion
for column in columns_to_convert:
    filtered_data[column] = filtered_data[column].apply(convert_value)

In [13]:
# Check for non-numeric entries in each column
for column in filtered_data.columns:
    # Convert to numeric, forcing non-convertible values to NaN
    numeric_data = pd.to_numeric(filtered_data[column], errors='coerce')
    
    # Find non-numeric entries
    non_numeric_entries = filtered_data[column][numeric_data.isna() & filtered_data[column].notna()]

    # Display unique non-numeric entries from this column
    if not non_numeric_entries.empty:
        print(f"Non-numeric entries in column '{column}':")
        print(non_numeric_entries.unique())

In [14]:
# In this dataset the missing data was replaced with zeros. As studies using dataset with zeros has already
# been done. I instead wanted to try a different approach. 
# First I made every zero cell into NaN values so that I can easily use imputation on them.
# This might not have been the best approach as now some columns that might've had actual zero values are also replaced.
# To see what would be the best way, we would need to do more data exploration.

In [15]:
# Replace zeros with NaN across all columns
filtered_data = filtered_data.replace(0, np.nan)

In [16]:
filtered_data.head()

,Nb,Ti,V,C,Si,Mn,P,S,Mo,Ni,...,Cr,Ce,Cu,Ca,Zr,O,Cb,YS(Mpa),UTS(Mpa),%EL
0,NaN,NaN,NaN,0.140,0.35,1.56,0.01,0.015,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.0,470.0,30.0
1,NaN,NaN,0.096,0.150,0.31,1.41,0.01,0.015,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378.0,579.0,23.0
2,NaN,0.075,NaN,0.054,0.03,0.54,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.0,NaN,NaN
3,NaN,0.092,NaN,0.061,0.03,0.56,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410.0,NaN,NaN
4,NaN,0.092,NaN,0.055,0.03,0.55,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395.0,NaN,NaN


In [17]:
# Ensure all columns are float64 before imputation
filtered_data = filtered_data.astype(float)

# Check for NaN values before imputation
print("NaN counts before imputation:", filtered_data.isna().sum())

# Check the data types to ensure they are all numeric
print("Data types:", filtered_data.dtypes)

NaN counts before imputation: Nb          264
Ti          411
V           422
C             0
Si           19
Mn            0
P           103
S           103
Mo          505
Ni          324
Al          153
N            76
Nb.1        265
V.1         422
B           653
Ti.1        411
Cr          496
Ce          710
Cu          442
Ca          717
Zr          714
O           696
Cb          736
YS(Mpa)     136
UTS(Mpa)    201
%EL         440
dtype: int64
Data types: Nb          float64
Ti          float64
V           float64
C           float64
Si          float64
Mn          float64
P           float64
S           float64
Mo          float64
Ni          float64
Al          float64
N           float64
Nb.1        float64
V.1         float64
B           float64
Ti.1        float64
Cr          float64
Ce          float64
Cu          float64
Ca          float64
Zr          float64
O           float64
Cb          float64
YS(Mpa)     float64
UTS(Mpa)    float64
%EL         float64
dtype: ob

In [18]:
# Re-apply the imputer
imputer = IterativeImputer(max_iter=10, random_state=0, sample_posterior=True, tol=0.001)
imputed_data = imputer.fit_transform(filtered_data)
filtered_data_imputed = pd.DataFrame(imputed_data, columns=filtered_data.columns)

# Check for NaN values after imputation
print("NaN counts after imputation:", filtered_data_imputed.isna().sum())

NaN counts after imputation: Nb          0
Ti          0
V           0
C           0
Si          0
Mn          0
P           0
S           0
Mo          0
Ni          0
Al          0
N           0
Nb.1        0
V.1         0
B           0
Ti.1        0
Cr          0
Ce          0
Cu          0
Ca          0
Zr          0
O           0
Cb          0
YS(Mpa)     0
UTS(Mpa)    0
%EL         0
dtype: int64


In [19]:
# Lastly we will normalize the dataset so that it is ready for training the surrogate models.

In [20]:
# Normalize the dataset
scaler = StandardScaler()
data_scaled = scaler.fit_transform(filtered_data_imputed)
preprocessed_data = pd.DataFrame(data_scaled, columns=filtered_data_imputed.columns)

In [21]:
preprocessed_data.tail()

,Nb,Ti,V,C,Si,Mn,P,S,Mo,Ni,...,Cr,Ce,Cu,Ca,Zr,O,Cb,YS(Mpa),UTS(Mpa),%EL
734,2.592111,1.593853,-0.008376,-0.585758,-0.953814,0.356633,-0.188852,-0.742888,0.672895,0.768998,...,-0.040494,-0.773474,0.515277,0.766389,-0.536978,1.336464,2.783529,1.678373,1.298288,0.067398
735,3.163172,0.239386,0.952984,0.165365,-0.953814,-0.219531,-0.161530,0.191500,-0.111356,-0.809993,...,-0.020024,0.571536,0.037617,0.761985,0.556072,-0.686066,0.568249,-0.080151,-0.241606,0.284455
736,8.057983,2.019112,-0.022008,-0.435533,-0.063547,-1.256627,0.528307,-0.122255,-0.068328,0.271583,...,-1.125052,2.012061,1.254711,-0.337461,0.014176,0.850157,-0.252455,0.803887,0.260440,-0.966519
737,17.439704,15.574239,5.805053,-0.135084,-1.423471,0.702332,0.163516,0.142470,1.938987,5.135789,...,4.730952,-0.659715,9.167737,0.884351,0.671528,-3.132126,-0.312138,1.840043,0.977476,-1.914025
738,2.347370,2.816472,0.189765,-0.735982,0.102467,0.788757,0.816664,-0.829168,-0.080744,1.058034,...,0.993083,0.591840,0.874679,-2.020819,0.324171,0.749137,0.311210,1.377080,0.955603,-0.604965


In [22]:
preprocessed_data.describe()

,Nb,Ti,V,C,Si,Mn,P,S,Mo,Ni,...,Cr,Ce,Cu,Ca,Zr,O,Cb,YS(Mpa),UTS(Mpa),%EL
count,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,...,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02,7.390000e+02
mean,5.768953e-17,3.845969e-17,-9.134176e-17,1.153791e-16,-3.076775e-16,7.691938e-17,-2.884477e-17,-1.153791e-16,-2.403730e-18,6.730445e-17,...,2.884477e-17,3.563530e-16,1.922984e-17,-1.682611e-17,2.403730e-16,-1.442238e-17,2.403730e-17,-6.778520e-16,6.586222e-16,9.614922e-17
std,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,...,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00,1.000677e+00
min,-4.452847e+00,-8.779645e-01,-5.664936e+00,-1.619303e+00,-2.253828e+00,-3.460456e+00,-3.978850e+00,-2.606977e+00,-1.572804e+01,-9.855287e+00,...,-2.466227e+00,-3.204495e+00,-2.652107e+00,-3.301248e+00,-3.189548e+00,-8.369085e+00,-3.422334e+00,-2.281002e+00,-2.428769e+00,-6.023799e+00
25%,-6.927138e-01,-3.869501e-01,-7.373329e-01,-6.158028e-01,-5.624332e-01,-2.339352e-01,-1.888516e-01,-7.428878e-01,-1.652758e-01,-5.311032e-01,...,-6.783104e-01,-6.577625e-01,-6.979389e-01,-6.666268e-01,-6.958874e-01,-5.471418e-01,-6.813416e-01,-4.710980e-01,-4.224271e-01,-6.169483e-01
50%,-5.924529e-02,-1.366685e-01,-7.440754e-02,-3.303761e-01,1.420519e-01,4.718662e-01,-1.068855e-01,-2.756938e-01,-1.101849e-01,-1.487151e-01,...,-2.844953e-01,5.871127e-03,-3.192321e-01,6.858773e-02,-5.308870e-02,8.418549e-02,-9.341683e-02,-2.102220e-01,-1.453628e-01,8.459281e-02
75%,3.486556e-01,1.839171e-01,4.926731e-01,6.160386e-01,6.899847e-01,6.447155e-01,2.402526e-03,3.590591e-01,2.993547e-01,2.115126e-01,...,3.645959e-01,5.964353e-01,6.233626e-01,6.938239e-01,6.366371e-01,5.973710e-01,6.276087e-01,3.812826e-01,3.504363e-01,6.175576e-01
max,1.743970e+01,1.557424e+01,8.452901e+00,4.371652e+00,2.646888e+00,1.163264e+00,1.814422e+01,4.462988e+00,3.612968e+00,5.135789e+00,...,6.021072e+00,3.119780e+00,9.167737e+00,2.901904e+00,4.434322e+00,3.536831e+00,8.837847e+00,9.290248e+00,9.099833e+00,2.866071e+00


In [23]:
has_zeros = (preprocessed_data == 0).any()
print(f"Column has zeros: {has_zeros}")

Column has zeros: Nb          False
Ti          False
V           False
C           False
Si          False
Mn          False
P           False
S           False
Mo          False
Ni          False
Al          False
N           False
Nb.1        False
V.1         False
B           False
Ti.1        False
Cr          False
Ce          False
Cu          False
Ca          False
Zr          False
O           False
Cb          False
YS(Mpa)     False
UTS(Mpa)    False
%EL         False
dtype: bool


In [24]:
# Check data types of each column
print(preprocessed_data.dtypes)

# Check for NaN values that might be interpreted differently
has_nans = preprocessed_data.isna().any()
print(f"Columns with NaNs: {has_nans}")

Nb          float64
Ti          float64
V           float64
C           float64
Si          float64
Mn          float64
P           float64
S           float64
Mo          float64
Ni          float64
Al          float64
N           float64
Nb.1        float64
V.1         float64
B           float64
Ti.1        float64
Cr          float64
Ce          float64
Cu          float64
Ca          float64
Zr          float64
O           float64
Cb          float64
YS(Mpa)     float64
UTS(Mpa)    float64
%EL         float64
dtype: object
Columns with NaNs: Nb          False
Ti          False
V           False
C           False
Si          False
Mn          False
P           False
S           False
Mo          False
Ni          False
Al          False
N           False
Nb.1        False
V.1         False
B           False
Ti.1        False
Cr          False
Ce          False
Cu          False
Ca          False
Zr          False
O           False
Cb          False
YS(Mpa)     False
UTS(Mpa)    Fal

In [25]:
preprocessed_data.to_excel('/Users/sepuliini/Desktop/DDO_kurssi/preprocessed_data.xlsx', 
                           index=False, engine='openpyxl')

In [23]:
# For task 2 and 3 I chose to use 5 different surrogate modelling techniques I've used before.

# Chosen modelling techniques:
 #   "RFR": RandomForestRegressor(n_estimators=100, random_state=42),
 #   "Ada": ensemble.AdaBoostRegressor(random_state=42),
 #   "GBR": GradientBoostingRegressor(random_state=42),
 #   "XGB": XGBRegressor(random_state=42),
 #   "SVM": svm.SVR()

# I made a script that would train these models with different hyperparameter setups to see which would perform best. 
# For comparison of models I used R2 score and MSE.

In [25]:
def evaluate_models_with_hyperparameters(data, targets, model_dict, param_grid):
    results = {}
    for target in targets:
        results[target] = {}
        X = data.drop(target, axis=1)
        y = data[target]
        
        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                            random_state=42)
        
        for model_name, model in model_dict.items():
            with warnings.catch_warnings():
                warnings.filterwarnings('error', category=FitFailedWarning)
                try:
                    grid_search = GridSearchCV(estimator=model, 
                                               param_grid=param_grid[model_name], 
                                               cv=3, scoring='neg_mean_squared_error', 
                                               verbose=0, n_jobs=-1)
                    grid_search.fit(X_train, y_train)
                except FitFailedWarning:
                    print(f"Fit failed for {model_name} on target {target}. Setting score to NaN.")
                    results[target][model_name] = {'MSE': float('nan'), 'R2': float('nan')}
                    continue
            
                # Best model
                best_model = grid_search.best_estimator_
                
                # Predict on the test set using the best model
                y_pred = best_model.predict(X_test)
                
                # Calculate MSE and R2 and store the results
                mse = mean_squared_error(y_test, y_pred)
                r2 = r2_score(y_test, y_pred)
                results[target][model_name] = {'MSE': mse, 'R2': r2}
                print(f'MSE for {target} using {model_name}: {mse}')
                print(f'R2 for {target} using {model_name}: {r2}')
    
    return results

# Define targets
targets = ['C', 'Si', 'Mn']

# Dictionary to hold models
model_dict = {
    "RFR": RandomForestRegressor(n_estimators=100, random_state=42),
    "Ada": ensemble.AdaBoostRegressor(random_state=42),
    "GBR": GradientBoostingRegressor(random_state=42),
    "XGB": XGBRegressor(random_state=42),
    "SVM": svm.SVR()
}

param_grid = {
    'RFR': {'n_estimators': [100, 200], 'max_depth': [10, 20, None]},
    'GBR': {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1], 
            'max_depth': [3, 5]},
    'XGB': {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1], 
            'max_depth': [3, 5]},
    'SVM': {'C': [1, 10], 'gamma': ['scale', 'auto']},
    'Ada': {'n_estimators': [50, 100, 200], 'learning_rate': [0.05, 0.1,
                                                              0.5]}
}

# Run evaluation
model_performance = evaluate_models_with_hyperparameters(preprocessed_data, 
                                                         targets, 
                                                         model_dict,
                                                         param_grid)

# Store best models and their parameters for each objective
best_models = {}
for target in targets:
    best_models[target] = []
    for model_name, model_info in model_performance[target].items():
        best_models[target].append({
            'model_name': model_name,
            'params': param_grid[model_name],
            'mse': model_info['MSE'],
            'r2': model_info['R2']
        })

# Print the best performing surrogate technique and its parameters for each objective
for target in targets:
    print(f"\nBest performing surrogate technique and parameters for '{target}':")
    for best_model in best_models[target]:
        print(f"Model: {best_model['model_name']}")
        print(f"Parameters: {best_model['params']}")
        print(f"MSE: {best_model['mse']}")
        print(f"R2: {best_model['r2']}")

MSE for C using RFR: 0.16539294817233632
R2 for C using RFR: 0.8146985560945407
MSE for C using Ada: 0.31856855275761736
R2 for C using Ada: 0.6430850682500104
MSE for C using GBR: 0.18307244678942078
R2 for C using GBR: 0.7948909605623735
MSE for C using XGB: 0.17982679464238752
R2 for C using XGB: 0.7985272947344535
MSE for C using SVM: 0.25643983758380295
R2 for C using SVM: 0.7126922719241535
MSE for Si using RFR: 0.18765754596556358
R2 for Si using RFR: 0.8337047740061336
MSE for Si using Ada: 0.30001094565593617
R2 for Si using Ada: 0.7341413170902134
MSE for Si using GBR: 0.1636136961654002
R2 for Si using GBR: 0.855011550750483
MSE for Si using XGB: 0.1895354928048299
R2 for Si using XGB: 0.8320406064799444
MSE for Si using SVM: 0.315408047064434
R2 for Si using SVM: 0.7204969712409584
MSE for Mn using RFR: 0.14320720385235805
R2 for Mn using RFR: 0.8460642184785689
MSE for Mn using Ada: 0.19762627737203925
R2 for Mn using Ada: 0.787568260268528
MSE for Mn using GBR: 0.13770875

In [27]:
# Now we can do optimization using interactive RVEA from DESDEO framework

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from desdeo_problem import Variable, ScalarObjective, MOProblem
from desdeo_emo.EAs.RVEA import RVEA
from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.exceptions import FitFailedWarning

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import ensemble

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


# Load dataset
data_full = '/Users/sepuliini/Desktop/DDO_kurssi/preprocessed_data.xlsx'
preprocessed_data = pd.read_excel(data_full, header=0)

# Define targets
targets = ['C', 'Si', 'Mn']
models = {}


# Train final models and save them
target = 'C'  # Focus on the 'C' target
X = preprocessed_data.drop(target, axis=1)
y = preprocessed_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)

# Train final models and save them
for target in targets:
    X = preprocessed_data.drop(target, axis=1)
    y = preprocessed_data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                        random_state=42)

    if target == 'C':
        # Obj 1 C:
        # Model: XGBRegressor
        # Parameters: {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 5]}
        param_grid = {
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5]
        }
        model_C = GridSearchCV(XGBRegressor(), param_grid, cv=5, n_jobs=-1)
        model_C.fit(X_train, y_train)
        models['C'] = model_C

    elif target == 'Si':
        # Obj 2 Si:
        # Model: GradientBoostingRegressor
        # Parameters: {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 5]}
        param_grid = {
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5]
        }
        model_Si = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=5, n_jobs=-1)
        model_Si.fit(X_train, y_train)
        models['Si'] = model_Si

    elif target == 'Mn':
        # Obj 3 Mn:
        # Model: RandomForestRegressor
        # Parameters: {'n_estimators': [100, 200], 'max_depth': [10, 20, None]}
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [10, 20, None]
        }
        model_Mn = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, n_jobs=-1)
        model_Mn.fit(X_train, y_train)
        models['Mn'] = model_Mn


print("Training done, proceeding to optimization")

def objective_1(x):
    return model_C.predict(x)

def objective_2(x):
    return -model_Si.predict(x)

def objective_3(x):
    return -model_Mn.predict(x)

# Define scalar objectives and variables
scalar_obj_1 = ScalarObjective("Objective 1", objective_1)  # Minimize C
scalar_obj_2 = ScalarObjective("Objective 2", objective_2)  # Maximize Si
scalar_obj_3 = ScalarObjective("Objective 3", objective_3)  # Maximize Mn
objectives = [scalar_obj_1, scalar_obj_2, scalar_obj_3]

variables = []
for col in X.columns:
    variable = Variable(col, lower_bound=0.3, upper_bound=1.0, initial_value=0.5)
    variables.append(variable)

# Initialize the multi-objective problem
problem = MOProblem(variables=variables, objectives=objectives)

# Initialize NSGA-III evolver
evolver = RVEA(problem, 
               interact=True,
                n_iterations=10,
                n_gen_per_iter=100,
                population_size=100)

evolver.population.objectives[evolver.population.non_dominated_fitness()]

evolver.set_interaction_type('Reference point')
pref, plot = evolver.start()

print(pref.content['message'])

#Please provide a reference point worse than the ideal point:
#Objective 1   -0.190433
#Objective 2    -0.59105
#Objective 3   -0.481949

# Lets make objectives 2 and 3 lil bit worse as
# we want to be sure to minimize objective 1 (C) as much as possible

response = evolver.population.ideal_fitness_val + [0.1, 0.5, 0.5]
pref.response = pd.DataFrame([response], columns = pref.content['dimensions_data'].columns)

pref, plot = evolver.iterate(pref)
print(pref.content['message'])

pref = None
pref, plot = evolver.iterate(pref)

print("finishing optimization")
while evolver.continue_evolution():
    evolver.iterate()
print("optimization done")

individuals, solutions, archive = evolver.end() 
print(solutions)



In [28]:
#And we get solutions:
optimization done
[[-0.23697636 -0.77931971 -0.58220169]
 [-0.23195221 -0.89401489 -0.52559354]
 [-0.38139722 -0.85689971 -0.38947469]
 [-0.22022802 -0.85719902 -0.53812511]
 [-0.24123307 -0.8111105  -0.56894991]
 [-0.31522438 -0.54667842 -0.43988908]
 [-0.38216463 -0.74474704 -0.39696482]
 [-0.38642135 -0.91049671 -0.35015146]
 [-0.33622929 -0.81522501 -0.42375647]
 [-0.22022802 -0.82172635 -0.58335402]
 [-0.16220675 -0.92195219 -0.53049093]
 [-0.38139722 -0.94373621 -0.34971934]
 [-0.33622929 -0.91049671 -0.39134722]
 [-0.21884048 -0.95327099 -0.51839148]
 [-0.22022802 -0.46469345 -0.59314882]
 [-0.24123307 -0.70531707 -0.58004108]
 [-0.09447157 -0.95299808 -0.5310671 ]
 [-0.2988877  -0.69569263 -0.51709511]
 [-0.17239802 -0.80186867 -0.59329286]
 [-0.33550832 -0.94736589 -0.34568619]
 [-0.29463103 -0.76832795 -0.46581647]
 [-0.25539884 -0.9211117  -0.49332833]
 [-0.28214434 -0.92138462 -0.46984963]
 [-0.23697636 -0.66170056 -0.58637888]
 [-0.26712301 -0.50216818 -0.54619141]
 [-0.28716847 -0.59587578 -0.52386504]
 [-0.24123307 -0.92420949 -0.51781532]
 [-0.33622929 -0.85789387 -0.39955756]
 [-0.24975453 -0.82135711 -0.53495621]
 [-0.23795323 -0.94978636 -0.50643607]
 [-0.17689948 -0.95299808 -0.52804223]
 [-0.20209214 -0.92417483 -0.5261697 ]
 [-0.38642135 -0.80517898 -0.38875448]
 [-0.24297738 -0.74139335 -0.56102765]
 [-0.286401   -0.95327099 -0.46034291]
 [-0.27960977 -0.74330654 -0.54460696]
 [-0.25228423 -0.85251229 -0.51896764]
 [-0.33197257 -0.66944475 -0.43888079]
 [-0.24869563 -0.41502027 -0.57571984]
 [-0.31522438 -0.45580298 -0.44939579]
 [-0.2988877  -0.86829556 -0.45371702]
 [-0.28716847 -0.70188922 -0.47762785]
 [-0.24975453 -0.78762314 -0.54763183]
 [-0.25037464 -0.95299808 -0.47359469]
 [-0.38642135 -0.64598072 -0.39293167]
 [-0.36591634 -0.4144558  -0.40992852]
 [-0.33622929 -0.76127446 -0.4378725 ]
 [-0.2788423  -0.77519292 -0.534236  ]
 [-0.286401   -0.81902871 -0.51997593]
 [-0.25539884 -0.64373705 -0.55123285]
 [-0.29708853 -0.94709298 -0.39638866]
 [-0.38216463 -0.53242854 -0.40114202]
 [-0.19783543 -0.89178276 -0.53754895]
 [-0.19783543 -0.84454823 -0.58162553]
 [-0.09447157 -0.8774745  -0.53826915]
 [-0.2704201  -0.87709618 -0.47287449]
 [-0.25670692 -0.82416479 -0.47618743]
 [-0.19783543 -0.69332727 -0.5958856 ]
 [-0.19783543 -0.77044058 -0.59199649]
 [ 0.16441855 -0.89737166 -0.53625258]
 [-0.21884048 -0.84482114 -0.57024628]
 [-0.23369652 -0.82966424 -0.54950436]]

SyntaxError: invalid syntax (1650817306.py, line 1)